# Quantum Pendulum

Authors: Rodrigo Sanchez, Julio Natividad, Leonardo Flores

TO DO

* **CHANGE ORDER OF VARIABLES IN FUNCTIONS. TO BE THE SAME AS THE EXPECTED ENERGY FUNCTION.**
* Graficar la solucion Phi(t), para diferentes valores de la energia. Phi(0) = 0.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
from scipy.special import mathieu_a, mathieu_b, mathieu_cem, mathieu_sem
# import seaborn as sns
from ipywidgets import interactive

# sns.set_style("darkgrid")
# sns.set_context("notebook")

%matplotlib inline

In [2]:
M = 1.0
g = 1.0
l = 1.0
U0 = M * g * l
h = 0.06
q = (4 * M * l**2 * U0) / h**2

## Functions

Respective energy for its respective Mathieu characterictic value

In [3]:
def energy(n, q):
    if n % 2 == 0:
        return (h**2 / (8 * M * l**2)) * mathieu_a(n, q) + U0
    else:
        return (h**2 / (8 * M * l**2)) * mathieu_b(n + 1, q) + U0

In [4]:
# Both, the mathieu_cem and mathieu_sem unpack two values, the
# result of their respective mathieu function and its derivative
# with respect to x (in degrees).


def ce(n, q, x):
    val, _ = mathieu_cem(n, q, x) / np.sqrt(np.pi)

    return val


def se(n, q, x):
    val, _ = mathieu_sem(n, q, x) / np.sqrt(np.pi)

    return val

# Keep an eye for larger values of n!


def f(n, q, x):
    if n % 2 == 0:
        return ce(n, q, (x - 180) / 2)
    else:
        return se(n + 1, q, (x - 180) / 2)

The parameter a is the height of the curve's peak, b is the position of the center of the peak and c (the standard deviation, sometimes called the Gaussian RMS width) controls the width of the "bell" - Wikipedia.

Coefficients:
$$
C_{n} = \exp\left(-\frac{(n - \bar{n})^{2}}{2 \sigma^2}\right)
$$

Gauss function:
$$
\Psi(\phi; \bar{n}) = \frac{1}{\sqrt{\sum |C_{n}|^{2}}} \sum_{n=0}^{\infty} C_{n} \psi_{n}(\phi)
$$

In [5]:
def gauss_coeff(n, nbar, sigma):
    return np.exp(- (n - nbar)**2 / (2 * sigma**2))

In [6]:
def norm(n_max, nbar, sigma):
    sum = 0

    for i in range(n_max + 1):
        sum += np.abs(gauss_coeff(i, nbar, sigma)**2)

    return 1 / np.sqrt(sum)

In [7]:
def f_gauss(n_max, q, x, nbar, sigma):
    sum = 0

    for i in range(n_max + 1):
        sum += f(i, q, x) * gauss_coeff(i, nbar, sigma)

    return norm(n_max, nbar, sigma) * sum

$$
\langle E \rangle_{\Psi} = \langle \Psi| \hat{H} |\Psi \rangle = \frac{1}{\sum |C_{n}|^{2}} \sum_{n=0}^{\infty}|C_{n}|^{2} E_{n}
$$

In [8]:
def expected_energy(nbar, n_max, sigma, q):
    num, den = 0, 0

    for i in range(n_max + 1):
        num += gauss_coeff(i, nbar, sigma)**2 * energy(i, q)
        den += gauss_coeff(i, nbar, sigma)**2

    return num / den

In [9]:
def pplot(n, nbar, sigma):
    x = np.arange(- np.pi, np.pi, 0.01)
    y = np.abs(f_gauss(n, q, x * 180 / np.pi, nbar, sigma))**2

    plt.plot(x, y)
    plt.show()

In [10]:
interactive_plot = interactive(
    # Ro says that sigma min must be 1. RO SAYS to verify dis.
    pplot, n=(0, 100, 1), nbar=(0.0, 40, 0.2), sigma=(1, 20, 0.1))
output = interactive_plot.children[-1]
output.layout.height = '270px'
interactive_plot

interactive(children=(IntSlider(value=50, description='n'), FloatSlider(value=20.0, description='nbar', max=40…

$$
\int_{0}^{2 \pi} |\Psi(\phi; \bar{n}) |^{2} d\theta = 1
$$

In [13]:
# n_max, q, x, nbar, sigma

def f_gauss_integral(n_max, q, nbar, sigma):
    def integrand(x):
        return f_gauss(n_max, q, x * 180 / np.pi, nbar, sigma)

    return integrate.quad(lambda x: np.abs(integrand(x))**2, 0, 2 * np.pi)[0]

In [14]:
f_gauss_integral(80, q, 0.2, 0.4)

1.000000000000002

## Test sub